<a href="https://colab.research.google.com/github/Nikuson123/Nikuson123/blob/main/Copy_of_RealTimeVoiceCloning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-Time Voice Cloning

This is a colab demo notebook using the open source project [CorentinJ/Real-Time-Voice-Cloning](https://github.com/CorentinJ/Real-Time-Voice-Cloning)
to clone a voice.

For other deep-learning Colab notebooks, visit [tugstugi/dl-colab-notebooks](https://github.com/tugstugi/dl-colab-notebooks).


Original issue: https://github.com/tugstugi/dl-colab-notebooks/issues/18

## Setup CorentinJ/Real-Time-Voice-Cloning

In [1]:
BRANCH = 'r1.5.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

  Cloning https://github.com/NVIDIA/NeMo.git (to revision r1.5.0) to /tmp/pip-install-f1_jivne/nemo-toolkit_31048c42476e4d5b914176319b9c2385
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-f1_jivne/nemo-toolkit_31048c42476e4d5b914176319b9c2385
  Running command git checkout -b r1.5.0 --track origin/r1.5.0
  Switched to a new branch 'r1.5.0'
  Branch 'r1.5.0' set up to track remote branch 'r1.5.0' from 'origin'.
     |████████████████████████████████| 12.7 MB 9.2 MB/s 
     |████████████████████████████████| 109 kB 72.1 MB/s 
     |████████████████████████████████| 1.2 MB 55.3 MB/s 
     |████████████████████████████████| 92 kB 1.1 MB/s 
     |████████████████████████████████| 235 kB 80.0 MB/s 
     |████████████████████████████████| 97 kB 9.3 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
  Using cached pytest_runner-5.3.1-py3-none-any.whl (7.1 kB)
     |████████████████████████████████| 1.7 MB 60.8 MB/s 
     |█████████████████████████

In [2]:
# Ignore pre-production warnings
import warnings
warnings.filterwarnings('ignore')
import nemo
# Import Speech Recognition collection
import nemo.collections.asr as nemo_asr
# Import Natural Language Processing colleciton
import nemo.collections.nlp as nemo_nlp
# Import Speech Synthesis collection
import nemo.collections.tts as nemo_tts
# We'll use this to listen to audio
import IPython

[NeMo W 2022-01-26 19:09:41 optimizers:50] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2022-01-26 19:09:45 experimental:28] Module <class 'nemo.collections.nlp.data.text_normalization.decoder_dataset.TextNormalizationDecoderDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-26 19:09:45 experimental:28] Module <class 'nemo.collections.nlp.data.text_normalization.tagger_dataset.TextNormalizationTaggerDataset'> is experimental, not ready for production and is not fully supported. Use at yo

In [3]:
#@title Setup CorentinJ/Real-Time-Voice-Cloning

#@markdown * clone the project
#@markdown * download pretrained models
#@markdown * initialize the voice cloning models

%tensorflow_version 1.x
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CorentinJ/Real-Time-Voice-Cloning.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q --recursive {git_repo_url}
  # install dependencies
  !cd {project_name} && pip install -q -r requirements.txt
  !pip install -q gdown
  !apt-get install -qq libportaudio2
  !pip install -q https://github.com/tugstugi/dl-colab-notebooks/archive/colab_utils.zip

  # download pretrained model
  #!cd {project_name} && wget https://github.com/blue-fish/Real-Time-Voice-Cloning/releases/download/v1.0/pretrained.zip && unzip -o pretrained.zip
  !cd {project_name} && mkdir -p saved_models/default/
  !cd {project_name}/saved_models/default/ && gdown https://drive.google.com/uc?id=1q8mEGwCkFy23KZsinbuvdKAQLqNKbYf1
  !cd {project_name}/saved_models/default/ && gdown https://drive.google.com/uc?id=1EqFMIbvxffxtjiVrtykroF6_mUh-5Z3s
  !cd {project_name}/saved_models/default/ && gdown https://drive.google.com/uc?id=1cf2NO6FtI0jDuy8AV3Xgn6leO6dHjIgu

import sys
sys.path.append(project_name)

from IPython.display import display, Audio, clear_output
from IPython.utils import io
import ipywidgets as widgets
import numpy as np
from dl_colab_notebooks.audio import record_audio, upload_audio

from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path

!ls 
encoder.load_model(project_name / Path("saved_models/default/encoder.pt"))
synthesizer = Synthesizer(project_name / Path("saved_models/default/synthesizer.pt"))
vocoder.load_model(project_name / Path("saved_models/default/vocoder.pt"))

TensorFlow is already loaded. Please restart the runtime to change versions.
     |████████████████████████████████| 15.3 MB 10.7 MB/s 
     |████████████████████████████████| 3.1 MB 55.0 MB/s 
     |████████████████████████████████| 8.3 MB 69.4 MB/s 
     |████████████████████████████████| 38.1 MB 342 kB/s 
     |████████████████████████████████| 86 kB 7.3 MB/s 
     |████████████████████████████████| 138 kB 66.7 MB/s 
     |████████████████████████████████| 676 kB 76.2 MB/s 
     |████████████████████████████████| 66 kB 5.1 MB/s 
     |████████████████████████████████| 59.9 MB 1.2 MB/s 
     |████████████████████████████████| 317 kB 72.7 MB/s 
     |████████████████████████████████| 1.1 MB 64.0 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
y

# *italicized text*

In [4]:
# Download audio sample which we'll try
# This is a sample from LibriSpeech Dev Clean dataset - the model hasn't seen it before
Audio_sample = '2086-149220-0033.wav'
!wget https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
# Listen to it
IPython.display.Audio(Audio_sample)

--2022-01-26 19:11:33--  https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
Resolving dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)... 52.219.92.170
Connecting to dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)|52.219.92.170|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 237964 (232K) [audio/wav]
Saving to: ‘2086-149220-0033.wav’

2086-149220-0033.wa 100%[===================>] 232.39K   908KB/s    in 0.3s    

2022-01-26 19:11:33 (908 KB/s) - ‘2086-149220-0033.wav’ saved [237964/237964]



In [5]:
# Speech Recognition model - QuartzNet
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5").cuda()
# Punctuation and capitalization model
punctuation = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained(model_name='punctuation_en_distilbert').cuda()
# Spectrogram generator which takes text as an input and produces spectrogram
spectrogram_generator = nemo_tts.models.FastPitchModel.from_pretrained(model_name="tts_en_fastpitch").cuda()
# Vocoder model which takes spectrogram and produces actual audio
vocoder = nemo_tts.models.HifiGanModel.from_pretrained(model_name="tts_hifigan").cuda()

[NeMo I 2022-01-26 19:11:37 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_quartznet15x5/versions/1.0.0rc1/files/stt_en_quartznet15x5.nemo to /root/.cache/torch/NeMo/NeMo_1.5.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2022-01-26 19:11:38 common:728] Instantiating model from pre-trained checkpoint


[NeMo W 2022-01-26 19:11:39 modelPT:131] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2022-01-26 19:11:39 modelPT:138] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2022-01-26 19:11:39 features:265] PADDING: 16
[NeMo I 2022-01-26 19:11:39 features:282] STFT using torch
[NeMo I 2022-01-26 19:11:40 save_restore_connector:149] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.5.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2022-01-26 19:11:40 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/punctuation_en_distilbert/versions/1.0.0rc1/files/punctuation_en_distilbert.nemo to /root/.cache/torch/NeMo/NeMo_1.5.0/punctuation_en_distilbert/613c4ee780c6fc158f49d3566cbd6636/punctuation_en_distilbert.nemo
[NeMo I 2022-01-26 19:11:46 common:728] Instantiating model from pre-trained checkpoint
[NeMo I 2022-01-26 19:11:48 tokenizer_utils:123] Getting HuggingFace AutoTokenizer with pretrained_model_name: distilbert-base-uncased, vocab_file: /tmp/tmpe0o2o6yl/tokenizer.vocab_file, special_tokens_dict: {}, and use_fast: False


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
[NeMo I 2022-01-26 19:12:12 features:265] PADDING: 1
[NeMo I 2022-01-26 19:12:12 features:282] STFT using torch
[NeMo I 2022-01-26 19:12:13 save_restore_connector:149] Model FastPitchModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.5.0/tts_en_fastpitch_align/b50e16c5d695b00855ae53d6ba4e4f7f/tts_en_fastpitch_align.nemo.
[NeMo I 2022-01-26 19:12:13 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_hifigan/versions/1.0.0rc1/files/tts_hifigan.nemo to /root/.cache/torch/NeMo/NeMo_1.5.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-01-26 19:12:21 common:728] Instantiating model from pre-trained checkpoint


[NeMo W 2022-01-26 19:12:24 modelPT:131] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-01-26 19:12:24 modelPT:138] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-01-26 19:12:24 features:265] PADDING: 0
[NeMo I 2022-01-26 19:12:24 features:282] STFT using torch


[NeMo W 2022-01-26 19:12:24 features:243] Using torch_stft is deprecated and will be removed in 1.1.0. Please set stft_conv and stft_exact_pad to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-01-26 19:12:24 features:265] PADDING: 0
[NeMo I 2022-01-26 19:12:24 features:282] STFT using torch
[NeMo I 2022-01-26 19:12:26 save_restore_connector:149] Model HifiGanModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.5.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


# ввв

In [6]:
#@title Record or Upload
#@markdown * Either record audio from microphone or upload audio from file (.mp3 or .wav) 

SAMPLE_RATE = 22050
record_or_upload = "Upload (.mp3 or .wav)" #@param ["Record", "Upload (.mp3 or .wav)"]
record_seconds =   9#@param {type:"number", min:1, max:10, step:1}

embedding = None
def _compute_embedding(audio):
  display(Audio(audio, rate=SAMPLE_RATE, autoplay=True))
  global embedding
  embedding = None
  embedding = encoder.embed_utterance(encoder.preprocess_wav(audio, SAMPLE_RATE))
def _record_audio(b):
  clear_output()
  audio = record_audio(record_seconds, sample_rate=SAMPLE_RATE)
  _compute_embedding(audio)
def _upload_audio(b):
  clear_output()
  audio = upload_audio(sample_rate=SAMPLE_RATE)
  _compute_embedding(audio)

if record_or_upload == "Record":
  button = widgets.Button(description="Record Your Voice")
  button.on_click(_record_audio)
  display(button)
else:
  #button = widgets.Button(description="Upload Voice File")
  #button.on_click(_upload_audio)
  _upload_audio("")

Saving audio_2022-01-26_19-16-34 (online-audio-converter.com).mp3_vocals (1) (1).mp3 to audio_2022-01-26_19-16-34 (online-audio-converter.com).mp3_vocals (1) (1).mp3


In [18]:
#@title Synthesize a text { run: "auto" }
text = "The various sub-fields of AI research are centered around particular goals and the use of particular tools" #@param {type:"string"}
  
def synthesize(embed, text):
  print("Synthesizing new audio...")
  #with io.capture_output() as captured:
  specs = synthesizer.synthesize_spectrograms([text], [embed])
  audio = vocoder.convert_spectrogram_to_audio(spec=specs[0])
  return audio.to('cpu').detach().numpy()
  clear_output()
  display(Audio(audio, rate=synthesizer.sample_rate, autoplay=True))

if embedding is None:
  print("first record a voice or upload a voice file!")
else:
  # Final result - with punctuation
   synthesize(embedding, text)

Synthesizing new audio...

| Generating 1/1


Done.



TypeError: ignored